# Prophet section


## module imports

In [2]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb 
from pylab import rcParams
import scipy
from scipy.stats import pearsonr

from prophet import Prophet
import json
from prophet.plot import plot_plotly, plot_components_plotly

from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
#                                       \/
#                   The Augmented Dickey-Fuller test can be used to test 
#                   whether a given time series is stationary or not
from sklearn.metrics import mean_squared_error
from tqdm import tqdm_notebook
#                    \/
#       responsible for displaying the progress bar
from itertools import product

import warnings
warnings.filterwarnings('ignore')
%matplotlib widget

In [20]:
plt.rcParams['figure.figsize'] = [10,10]

## calling the csv data

In [ ]:
temp2 = pd.read_csv('/Users/stephenshaeffer/Desktop/TXT-CSV/temperature_sation2.csv')
temp2.head()

,y,ds
0,67,2023-04-12
1,66,2023-04-12
2,66,2023-04-12
3,66,2023-04-12
4,66,2023-04-12


## defining the "m" method

In [ ]:
m = Prophet()
m.fit(temp2)

11:26:29 - cmdstanpy - INFO - Chain [1] start processing
11:26:30 - cmdstanpy - INFO - Chain [1] done processing
11:26:30 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
11:26:30 - cmdstanpy - INFO - Chain [1] start processing
11:26:40 - cmdstanpy - INFO - Chain [1] done processing


## defining prediction dataframes

In [ ]:
tempDayfuture = m.make_future_dataframe(periods=1)
tempDayfuture.tail()

In [ ]:
tempWeekfuture = m.make_future_dataframe(periods=7)
tempWeekfuture

In [ ]:
temp2Weekfuture = m.make_future_dataframe(periods=14)
temp2Weekfuture

## utilizing the prediction data frames with the "predict" method

In [ ]:
tempDayforecast = m.predict(tempDayfuture)
tempDayforecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [ ]:
tempWeekforecast = m.predict(tempWeekfuture)
tempWeekforecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [ ]:
temp2Weekforecast = m.predict(temp2Weekfuture)
temp2Weekforecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

## plotting the 24 hour forcasting

In [ ]:
# forcast 1
m.plot(tempDayforecast)

In [ ]:
# forcast 2
m.plot_components(tempDayforecast)

In [ ]:
# forcast 3
plot_plotly(m, tempDayforecast)

In [ ]:
# forcast 4
plot_components_plotly(m, tempDayforecast)

## plotting the weekly forcasting


In [ ]:
# forcast 1
m.plot(tempWeekforecast)

In [ ]:
# forcast 2
m.plot_components(tempDayforecast)

In [ ]:
# forcast 3
plot_plotly(m, tempDayforecast)

In [ ]:
# forcast 4
plot_components_plotly(m, tempDayforecast)

## plotting the 2 week forcasting


In [1]:
# forcast 1
m.plot(temp2Weekforecast)

NameError: name 'm' is not defined

In [ ]:
# forcast 2
m.plot_components(temp2Weekforecast)

In [ ]:
# forcast 3
plot_plotly(m, temp2Weekforecast)

In [ ]:
# forcast 4
plot_components_plotly(m, temp2Weekforecast)

# ARIMA section


## plotting the data


In [ ]:
plt.plot(temp2['ds'], temp2['y'])
plt.ylabel('Temperature (F)')
plt.xlabel('Date')
plt.xticks(rotation=30)
plt.rcParams['figure.figsize'] = [10, 10]

## testing for stationary

In [ ]:
"""
testing for unit root with ACF

"""
def stationarity_test1():
# a1 and c1 are for the ranges of p that should be acceptable
    a1 = 0.001
    a2 = 0.01
    b1 = 0.05
    b2 = 0.001
    c1 = 0.10

    ad_fuller_result = adfuller(temp2['y'])

    p = (ad_fuller_result[1])
    
    ADF1 = (ad_fuller_result[0])
    p1 = (ad_fuller_result[1])

    temp2['log_diff'] = np.log(temp2['y']) # taking that log of the data 
    temp2['log_diff'] = temp2['log_diff'].diff() # taking the difference of the log data
    ad_fuller_result2 = adfuller(temp2['log_diff'][1:])

    ADF2 = (ad_fuller_result2[0])
    p2 = (ad_fuller_result2[1])

    if ((ADF2 and p2) < (ADF1 and p1)) and ((p2 < a1) or (a1 < p2 < a2) or (b1 < p2 < b2)):
        print('this is stationary\n ADF                p-value')
        return ADF2, p2
    elif (ADF2 and p2) > (ADF1 and p1) and ((p1 < a1) or (a1 < p1 < a2) or (b1 < p1 < b2)):
        print('this is stationary\n ADF                p-value')
        return ADF2, p2

stationarity_test1()

## determining best parameters with AIC and the fitting then best model

In [ ]:
"""
AR: order of 0-10
I: equals 1 (since we only difference once)
MA: order of 0-10
"""
def optimize_ARIMA(endog, order_list):
    """
        Return dataframe with parameters and corresponding AIC (Akaike Information Criterion)
                                                            ^
                                            This is an estimator of prediction error
        order_list - list with (p, d, q) tuples
        endog - the observed variable
    """
    
    results = []
    
    for order in tqdm_notebook(order_list):
        try: 
            model = SARIMAX(endog, order=order, simple_differencing=False).fit(disp=False)
        except:
            continue
            
        aic = model.aic
        results.append([order, model.aic])
        
    result_df = pd.DataFrame(results)
    result_df.columns = ['(p, d, q)', 'AIC']
    #Sort in ascending order, lower AIC is better
    result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)
    
    return result_df

In [ ]:
ps = range(0, 10, 1)
d = 1
qs = range(0, 10, 1)

# Create a list with all possible combination of parameters
parameters = product(ps, qs)
parameters_list = list(parameters)

order_list = []

for each in parameters_list:
    each = list(each)
    each.insert(1, 1)
    each = tuple(each)
    order_list.append(each)

In [ ]:
result_df = optimize_ARIMA(temp2['y'], order_list)
result_df

In [ ]:
"""
fitting the best (p, d, q) Seasonal Auto Regressive Integrated Moving Average 
"""
best_model1 = SARIMAX(temp2['y'], order=(3, 1, 6), simple_differencing=False)
#                                                           ^
#                             Whether or not to use partially conditional maximum likelihood estimation
res = best_model1.fit(disp=False)
print(res.summary())

In [ ]:
res.plot_diagnostics();

## evaluating forecasting results of the model

In [ ]:
"""
forecasting a predicted mean value
"""
n_forecast = 1000
predict = res.get_prediction(end=best_model1.nobs + n_forecast)
#                                            /\
#                           this means number of observations 
idx1 = np.arange(len(predict.predicted_mean))

fig, ax = plt.subplots()
ax.plot(temp2['y'], 'blue')
ax.plot(idx1[-n_forecast:], predict.predicted_mean[-n_forecast:], 'y--')

ax.set(title = 'Temperature Forecast 1')
plt.show()

In [ ]:
temp2['model'] = predict.predicted_mean
temp2

## taking into account mean squared error value for the model

In [ ]:
mse = mean_squared_error(temp2['y'], temp2['model'])
print(f'MSE: {mse}')

# SARIMA section

## testing for stationary

In [ ]:
"""
data_tr_2 is the seasonal difference
data_tr_1 is the log difference 
"""
temp2['seasonal_diff'] = temp2['log_diff'][1:].diff(1)
temp2.head(20)

In [ ]:
def stationarity_test2():
    
    a1 = 0.001
    a2 = 0.01
    b1 = 0.05
    b2 = 0.001
    c1 = 0.10

    ad_fuller_result = adfuller(temp2['y'])

    p = (ad_fuller_result[1])
    
    ADF1 = (ad_fuller_result[0])
    p1 = (ad_fuller_result[1])

    temp2['seasonal_diff'] = np.log(temp2['y']) # taking that log of the data 
    temp2['seasonal_diff'] = temp2['seasonal_diff'].diff() # taking the difference of the log data
    ad_fuller_result2 = adfuller(temp2['seasonal_diff'][2:])

    ADF2 = (ad_fuller_result2[0])
    p2 = (ad_fuller_result2[1])

    if ((ADF2 and p2) < (ADF1 and p1)) and ((p2 < a1) or (a1 < p2 < a2) or (b1 < p2 < b2)):
        print('this is stationary\n ADF                p-value')
        return ADF2, p2
    elif (ADF1 and p1) > (ADF1 and p1) and ((p1 < a1) or (a1 < p1 < a2) or (b1 < p1 < b2)):
        print('this is stationary\n ADF                p-value')
        return ADF1, p1

stationarity_test2()

## determining best parameters with AIC and then fitting the best model

In [ ]:
def optimize_SARIMA(endog, parameters_list, d, D, s):
    """
        Returns dataframe with parameters and correspondings AIC

        endog           - the observed variable 
        parameters_list - list with (p, q, P, Q) tuples 
        d               - integration order 
        D               - seasonal integration order 
        s               - length of a season
    """

    results = []

    for param in tqdm_notebook(parameters_list):
        try:
            model2 = SARIMAX(endog,
            order = (param[0], d, param[1]),
            seasonal_order = (param[2], D, param[3], s),
            simple_differencing=False).fit(disp=False)
        except:
            continue

        aic = model2.aic
        results.append([param, aic])

    result_df = pd.DataFrame(results)
    result_df.columns = ['(p,q)x(P,Q)', 'AIC']

    result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)

    return result_df

In [ ]:
""" the middle value for the range is 2 because it is representing 1/5th of the data (to save computation time) """
p = range(0,2,1)
d = 1
q = range(0,2,1)
P = range(0,2,1)
D = 1
Q = range(0,2,1)
s = 4

parameters = product(p, q, P, Q)
parameters_list = list(parameters)

print(len(parameters_list))
print(parameters_list[:5])

In [ ]:
#                                                         order of differencing 
#                                                         |
#                                                         | seasonal order of differencing
#                                                         |  |  
#                                                         |  |  length of the season
#                                                         |  |  |
#                                                        \/ \/ \/
result_df = optimize_SARIMA(temp2['y'], parameters_list, 1, 1, 2)
result_df

## fitting the best model fit with SARIMAX 

In [ ]:
best_model2 = SARIMAX(temp2['y'], order=(2,1,3), seasonal_order=(0,1,0,4), simple_differencing=False)
res = best_model2.fit(disp=False)

print(res.summary())

In [ ]:
res.plot_diagnostics();

In [ ]:
n_forecast = 1000
predict = res.get_prediction(end=best_model2.nobs + n_forecast)
idx = np.arange(len(predict.predicted_mean))

fig, ax = plt.subplots()
ax.plot(temp2['y'], 'blue')
ax.plot(idx[-n_forecast:], predict.predicted_mean[-n_forecast:], 'g--')

ax.set(title='Temperature Forcast 2')
plt.show()

In [ ]:
temp2['model2'] = predict.predicted_mean
temp2

## taking into account mean squared error value for the model

In [ ]:
mse = mean_squared_error(temp2['y'], temp2['model2'])
print(f'MSE: {mse}')

# ARIMA and SARIMA comparison

In [ ]:
plt.plot(temp2['y'], color='blue', label='actual')
plt.plot(temp2['model'], color='red', label='model')
plt.plot(temp2['model2'], color='green', label='model2')
plt.legend(loc='best')
plt.title('comparison of ARIMA and SARIMA model to actual')
plt.show()